In [10]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold
random_num = np.random.randint(10000)

random_num = np.random.randint(10000)

boston = load_boston()
features = boston.feature_names
x_data = boston.data
y_data = boston.target
print('features:', features)
print('x_data.shape: \t', x_data.shape)
print('y_data.shape: \t', y_data.shape)

features: ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
x_data.shape: 	 (506, 13)
y_data.shape: 	 (506,)


In [15]:
x_data = pd.DataFrame(x_data, columns=features)
y_data = pd.Series(y_data, index=x_data.index)

train_idx, val_idx = train_test_split(list(range(len(x_data))), test_size=0.5, random_state=random_num, shuffle=True)
train1_idx, train2_idx = train_test_split(train_idx, test_size=0.5, random_state=random_num, shuffle=True)

x_train = x_data.loc[train_idx]
x_train_1 = x_data.loc[train1_idx]
x_train_2 = x_data.loc[train2_idx]
x_val = x_data.loc[val_idx]

y_train = y_data.loc[train_idx]
y_train_1 = y_data.loc[train1_idx]
y_train_2 = y_data.loc[train2_idx]
y_val = y_data.loc[val_idx]

xg_train_0 = xgb.DMatrix(x_train, label=y_train)
xg_train_1 = xgb.DMatrix(x_train_1, label=y_train_1)
xg_train_2 = xgb.DMatrix(x_train_2, label=y_train_2)
xg_val = xgb.DMatrix(x_val, label=y_val)

params = {'objective': 'reg:linear', 'verbose': False}
model_0 = xgb.train(params, xg_train_0, 30)
model_1 = xgb.train(params, xg_train_1, 30)
model_1.save_model('model_1.model')
model_2_v1 = xgb.train(params, xg_train_2, 30)
model_2_v2 = xgb.train(params, xg_train_2, 30, xgb_model=model_1)

params.update({'process_type': 'update',
               'updater'     : 'refresh',
               'refresh_leaf': True})
model_2_v2_update = xgb.train(params, xg_train_2, 30, xgb_model=model_1)

print('full train\t',mse(model_0.predict(xg_val), y_val)) # benchmark
print('model 1 \t',mse(model_1.predict(xg_val), y_val))  
print('model 2 \t',mse(model_2_v1.predict(xg_val), y_val))  # "before"
print('model 1+2\t',mse(model_2_v2.predict(xg_val), y_val))  # "after"
print('model 1+update2\t',mse(model_2_v2_update.predict(xg_val), y_val))  # "after"

full train	 13.0713925453
model 1 	 23.8455878457
model 2 	 17.6584998215
model 1+2	 21.150640958
model 1+update2	 20.9334575737
